In [7]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  8 19:40:08 2021

@author: Nooreldean Koteb
"""
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from collections import defaultdict 
from sklearn.model_selection import train_test_split

class SAE(nn.Module):
    def __init__(self, inp, hidden, layers=2):
        super(SAE, self).__init__()
        self.line1 = nn.Linear(inp, hidden)
        self.line2 = nn.Linear(hidden, hidden//2)
        self.line3 = nn.Linear(hidden//2, hidden)
        # self.line4 = nn.Linear(hidden//4, hidden//2)
        # self.line5 = nn.Linear(hidden//2, hidden)
        # self.line6 = nn.Linear(hidden//4, hidden//2)
        # self.line7 = nn.Linear(hidden//2, hidden)

        self.line4 = nn.Linear(hidden, inp)
        
        self.activation = nn.Sigmoid()
        
        self.dropout = nn.Dropout(0.1)
        

    def forward(self, x):
        x = self.activation(self.line1(x))
        # x = self.dropout(x)
        
        x = self.activation(self.line2(x))
        # x = self.dropout(x)
        
        x = self.activation(self.line3(x))
        # x = self.dropout(x)
        
        # x = self.activation(self.line4(x))
        # x = self.dropout(x)
        
        # x = self.activation(self.line5(x))
        # x = self.dropout(x)
        
        # x = self.activation(self.line6(x))
        # x = self.dropout(x)
        
        # x = self.activation(self.line7(x))
        # # x = self.dropout(x)
        
        x = self.line4 (x)
        
        return x

#Use GPU if available
def use_gpu():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}\n')
    return device

#Used For default dictionary
def def_value():
    return 'None'



def preprocess(train_data, test_data):
    #UserID, MovieID
    test = np.array(pd.read_csv(test_data, sep = ' ',  header = 0), dtype=int)
    #UserID, MovieID, Rating
    train = np.array(pd.read_csv(train_data, dtype=float, sep = ' ', header = 0, encoding = 'latin-1'), dtype=float)
    
    users_dic = defaultdict(def_value)
    movies_dic = defaultdict(def_value)
    
    users_list = list(train[:, 0])+list(test[:, 0])
    movies_list = list(train[:, 1])+list(test[:, 1])
    
    key1 = 0
    for i in users_list:
        if i not in users_dic.keys():
            key1+=1
            users_dic[i] = key1
    

    key2 = 0
    for i in movies_list:
        if i not in movies_dic.keys():
            key2+=1
            movies_dic[i] = key2
    
    
    n_users = key1       
    n_movies = key2
    
    return (test, train, users_dic, movies_dic, users_list, movies_list, n_users, n_movies)
    
#Create User Profiles from data
def profile(dat, device, n_movies, movies_dic, user_dic):
    profiles = []
    for user in users_dic.keys():
        movies = dat[:, 1][dat[:, 0] == user]
        ratings = dat[:, 2][dat[:, 0] == user]
        
        movie_keys = []
        for i in movies:
            movie_keys.append(movies_dic[i])
        
        profile = np.zeros(n_movies)
        profile[movie_keys] = ratings
        
        
        profiles.append(profile)
    return torch.tensor(profiles, dtype=torch.float, device=device)
    

def train_model(device, n_users, n_movies, X_train, 
          epochs=50, layers=2, hidden=20, optimizer=('RMS', 0.01, 0.5)):
    
    model = SAE(n_movies, hidden=hidden, layers=layers)
    if device.type == 'cuda':
        model.cuda()
    

    criterion = nn.MSELoss()
    optimizer = optim.RMSprop(model.parameters(), lr = optimizer[1], weight_decay = optimizer[2])#


    for epoch in range(1, epochs + 1):
        RMSE = 0
        step = 0
        
        #Loop through all users
        for user in range(n_users):
            expected = Variable(X_train[user]).unsqueeze(0).clone()
            
            if torch.sum(expected.data > 0) > 0:
                result = model(Variable(X_train[user]).unsqueeze(0))
                
                expected.require_grad = False

                result[expected == 0] = 0                
                loss = criterion(result, expected)
                rated = expected.data > 0
                mean_corrector = n_movies/float(torch.sum(rated) + .00000000001)
                
                loss.backward()
                
                #RMSE
                RMSE += torch.sqrt(loss.data * mean_corrector)
                optimizer.step()
            
        print('epoch: '+str(epoch)+'/'+str(epochs)+' train loss: '+str(RMSE/step))        
    return model

# def test_model(model, n_users, n_movies, X_train, X_test):
#     RMSE = 0
#     step = 0

#     for user in range(n_users):
#         expected = Variable(X_test[user]).unsqueeze(0).clone()
        
#         if torch.sum(expected.data > 0) > 0:
#             result = model(Variable(X_train[user]).unsqueeze(0))
            
#             expected.require_grad = False

#             result[expected == 0] = 0                
#             loss = criterion(result, expected)
#             rated = expected.data > 0
#             mean_corrector = n_movies/float(torch.sum(rated) + .00000000001)
            
#             loss.backward()
            
#             #RMSE
#             RMSE += torch.sqrt(loss.data * mean_corrector)
#             optimizer.step()
    
#     #Print normalized test loss
#     print('test loss: '+str(RMSE/step))        
#     return test_loss/s
    
def pred_model(model,X_train, n_users, pred_dat, users_dic, movies_dic):    
    #Pred results
    raw_results = []
    for user in range(n_users):
        result = model(Variable(X_train[user]).unsqueeze(0))
        raw_results.append(result.data.cpu().numpy())
    
    
    results = []
    t = 0
    for i in pred_dat:
        try:
            results.append(f'{raw_results[users_dic[i[0]]][0][movies_dic[i[1]]] :.1f}') 
        except:
            t+=1
            results.append('2.5')
    
    return results
    
def write(results, outfile):      
    with open(outfile,"w") as f:
        for i in results:
            print(i)
            f.write(f'{i}\n')  
            
        f.close()        

In [2]:
device = use_gpu()

Using device: cuda



In [3]:
pred_dat, train, users_dic, movies_dic, users_list, movies_list, n_users, n_movies = preprocess('additional_files/train.dat', 'test.dat')

In [4]:
X_train = profile(train, device, n_movies, movies_dic, users_dic)

X_train

tensor([[0.0000, 1.0000, 4.5000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 5.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 2.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 4.5000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0')

In [5]:
model = train_model(device, n_users, n_movies, X_train, epochs=100, layers=2, 
                    hidden=20, optimizer=('RMS', 0.00001, 0.5))



results = pred_model(model, X_train, n_users, pred_dat, users_dic, movies_dic)

results

epoch: 1/100 train loss: tensor(inf, device='cuda:0')
epoch: 2/100 train loss: tensor(inf, device='cuda:0')
epoch: 3/100 train loss: tensor(inf, device='cuda:0')
epoch: 4/100 train loss: tensor(inf, device='cuda:0')
epoch: 5/100 train loss: tensor(inf, device='cuda:0')
epoch: 6/100 train loss: tensor(inf, device='cuda:0')
epoch: 7/100 train loss: tensor(inf, device='cuda:0')
epoch: 8/100 train loss: tensor(inf, device='cuda:0')
epoch: 9/100 train loss: tensor(inf, device='cuda:0')
epoch: 10/100 train loss: tensor(inf, device='cuda:0')
epoch: 11/100 train loss: tensor(inf, device='cuda:0')
epoch: 12/100 train loss: tensor(inf, device='cuda:0')
epoch: 13/100 train loss: tensor(inf, device='cuda:0')
epoch: 14/100 train loss: tensor(inf, device='cuda:0')
epoch: 15/100 train loss: tensor(inf, device='cuda:0')
epoch: 16/100 train loss: tensor(inf, device='cuda:0')
epoch: 17/100 train loss: tensor(inf, device='cuda:0')
epoch: 18/100 train loss: tensor(inf, device='cuda:0')
epoch: 19/100 train

['2.9',
 '3.5',
 '2.9',
 '3.1',
 '3.8',
 '2.8',
 '2.9',
 '4.3',
 '3.9',
 '3.7',
 '3.4',
 '3.7',
 '4.0',
 '3.4',
 '4.1',
 '4.0',
 '3.0',
 '3.1',
 '4.0',
 '3.8',
 '4.0',
 '4.6',
 '3.5',
 '3.3',
 '3.3',
 '3.7',
 '3.9',
 '3.4',
 '3.8',
 '3.4',
 '3.0',
 '3.0',
 '2.8',
 '4.0',
 '3.0',
 '3.4',
 '3.5',
 '3.9',
 '3.8',
 '3.6',
 '1.6',
 '2.0',
 '4.7',
 '2.4',
 '2.3',
 '2.8',
 '3.7',
 '2.3',
 '2.9',
 '1.5',
 '4.2',
 '3.7',
 '4.6',
 '3.8',
 '3.6',
 '3.5',
 '3.6',
 '2.7',
 '3.2',
 '4.0',
 '4.1',
 '4.2',
 '3.1',
 '4.0',
 '5.5',
 '4.4',
 '4.2',
 '3.9',
 '2.2',
 '4.0',
 '4.0',
 '4.4',
 '3.5',
 '3.8',
 '4.1',
 '4.6',
 '5.3',
 '4.2',
 '1.8',
 '5.4',
 '2.4',
 '3.7',
 '3.6',
 '4.0',
 '3.4',
 '3.3',
 '3.1',
 '3.7',
 '3.7',
 '3.3',
 '4.6',
 '5.4',
 '3.1',
 '1.6',
 '4.0',
 '3.6',
 '3.0',
 '3.3',
 '3.8',
 '2.8',
 '4.0',
 '5.3',
 '3.7',
 '4.2',
 '3.8',
 '3.8',
 '2.1',
 '4.2',
 '4.0',
 '4.2',
 '3.2',
 '3.4',
 '3.5',
 '5.5',
 '3.4',
 '1.8',
 '4.6',
 '3.8',
 '3.8',
 '4.3',
 '3.3',
 '3.7',
 '2.9',
 '3.0',
 '3.2',


In [8]:
write(results, '7.txt')

2.9
3.5
2.9
3.1
3.8
2.8
2.9
4.3
3.9
3.7
3.4
3.7
4.0
3.4
4.1
4.0
3.0
3.1
4.0
3.8
4.0
4.6
3.5
3.3
3.3
3.7
3.9
3.4
3.8
3.4
3.0
3.0
2.8
4.0
3.0
3.4
3.5
3.9
3.8
3.6
1.6
2.0
4.7
2.4
2.3
2.8
3.7
2.3
2.9
1.5
4.2
3.7
4.6
3.8
3.6
3.5
3.6
2.7
3.2
4.0
4.1
4.2
3.1
4.0
5.5
4.4
4.2
3.9
2.2
4.0
4.0
4.4
3.5
3.8
4.1
4.6
5.3
4.2
1.8
5.4
2.4
3.7
3.6
4.0
3.4
3.3
3.1
3.7
3.7
3.3
4.6
5.4
3.1
1.6
4.0
3.6
3.0
3.3
3.8
2.8
4.0
5.3
3.7
4.2
3.8
3.8
2.1
4.2
4.0
4.2
3.2
3.4
3.5
5.5
3.4
1.8
4.6
3.8
3.8
4.3
3.3
3.7
2.9
3.0
3.2
3.9
2.7
3.0
3.2
4.3
3.1
3.8
3.6
4.0
3.6
3.6
4.0
5.0
2.7
3.7
4.3
1.5
2.7
5.5
3.2
3.2
3.4
3.4
3.6
3.9
3.8
4.0
3.4
3.7
3.0
1.6
3.3
2.8
2.9
5.8
3.2
5.3
3.5
4.0
3.5
3.1
2.3
4.0
3.8
3.4
4.2
3.6
4.3
3.6
5.3
3.7
3.0
2.0
3.5
4.6
3.4
3.8
2.0
2.8
3.5
3.9
4.7
4.3
3.6
3.9
2.7
3.7
3.0
4.0
3.4
3.4
3.3
3.7
3.8
1.8
4.0
5.1
2.3
1.7
2.6
4.3
1.6
3.1
4.6
3.3
3.5
3.4
3.0
3.3
3.5
3.1
3.1
3.0
4.3
4.1
3.2
3.2
3.2
3.5
3.8
2.6
3.7
3.2
3.6
3.8
3.1
3.7
2.5
3.6
3.9
3.6
3.7
3.3
3.5
3.6
3.2
4.6
3.1
3.3
2.2
2.5
3.1
3.2
2.6
4.3


3.6
2.7
3.3
3.0
3.5
3.5
3.1
3.1
2.3
3.1
2.2
3.3
2.8
3.1
3.2
3.8
3.7
2.3
3.7
3.2
3.0
3.3
2.9
3.7
3.1
3.6
2.3
2.1
3.1
3.8
3.3
3.4
3.5
3.0
3.1
2.8
3.3
2.4
3.1
3.1
4.6
3.2
2.6
3.5
2.7
2.8
2.1
2.4
3.5
1.8
5.0
4.0
3.2
3.1
3.7
4.9
4.2
4.1
3.8
4.0
4.0
3.4
2.9
5.5
3.3
3.2
4.0
3.4
2.7
2.9
3.7
3.9
3.7
2.8
3.3
3.3
2.7
5.3
4.0
3.7
3.8
3.8
3.4
3.5
3.5
2.1
3.1
3.9
3.2
3.3
3.3
3.7
3.8
3.6
3.5
3.7
3.5
2.3
3.4
3.4
2.3
5.6
3.5
3.2
3.9
3.1
3.5
3.8
2.2
3.5
3.7
4.0
3.8
3.7
3.0
2.6
2.3
3.1
3.5
3.1
3.5
1.8
3.0
2.5
3.3
4.3
3.6
4.2
3.1
2.6
3.3
3.6
3.1
3.0
3.5
4.3
3.5
3.0
2.1
3.2
3.4
3.3
3.6
3.7
3.5
3.3
3.6
3.0
3.7
3.6
4.3
4.6
3.1
3.6
3.5
3.1
3.1
3.5
3.8
3.7
2.6
3.5
3.1
3.3
3.1
3.4
2.9
3.8
3.2
3.7
2.5
3.3
3.8
3.1
3.4
3.7
2.8
3.1
2.9
4.0
3.6
3.5
5.0
4.2
3.3
4.8
5.1
4.1
3.7
3.9
3.2
4.4
3.0
2.2
3.7
2.9
3.6
2.8
3.6
3.2
3.8
2.5
2.7
3.0
3.7
2.2
3.8
3.1
3.3
3.7
3.8
3.6
3.1
2.0
2.5
3.7
2.8
4.0
3.6
3.7
2.0
3.4
3.9
2.1
2.2
2.4
3.4
2.5
2.9
3.7
2.3
3.3
2.5
3.3
3.5
3.6
3.6
2.8
3.5
3.4
3.0
4.0
3.4
3.6
3.7
2.2
2.3
2.9
2.1
2.7


3.3
2.8
3.3
3.2
3.7
3.1
2.4
3.1
2.2
4.0
3.8
3.8
4.0
4.0
3.0
3.7
5.6
3.6
4.4
4.6
3.7
3.0
2.6
2.1
2.2
3.4
2.5
3.4
3.7
3.1
2.4
3.1
3.8
3.0
3.3
3.3
3.4
3.2
3.2
2.9
4.2
2.5
4.0
2.7
3.9
3.5
3.2
3.4
4.0
3.2
3.9
3.8
3.4
3.5
4.0
3.9
4.9
5.3
4.1
3.8
4.1
4.0
3.8
3.9
3.7
3.5
3.2
3.6
2.7
2.8
3.6
3.0
3.7
3.3
3.1
3.6
3.6
3.6
3.6
3.7
1.5
3.3
3.5
3.9
3.2
3.8
3.5
2.8
3.3
3.4
3.9
3.8
3.4
4.0
3.2
4.0
3.9
3.4
3.3
3.5
3.8
3.5
3.8
3.7
4.1
4.0
3.7
3.4
4.0
3.7
2.7
4.0
3.8
3.8
2.1
3.4
3.6
3.9
3.9
2.7
3.6
1.9
3.1
3.5
3.1
3.3
2.9
3.0
3.7
3.6
3.6
3.1
2.8
4.2
4.3
1.9
3.4
3.0
3.4
4.0
3.5
2.5
3.8
3.7
4.2
3.4
3.6
1.9
3.5
3.8
3.8
3.8
4.0
3.4
3.3
3.3
4.0
2.9
3.8
3.2
2.6
3.8
3.6
2.4
3.8
4.0
3.1
3.1
3.8
3.0
3.7
3.4
3.9
3.8
3.5
3.3
3.4
3.1
2.0
3.1
4.7
3.2
3.8
2.2
3.0
3.6
3.1
3.0
3.7
3.8
3.4
3.3
5.0
3.5
2.6
2.7
4.0
5.4
4.1
4.6
4.0
4.3
3.8
3.9
3.2
3.3
3.8
5.0
3.4
3.6
3.8
3.6
3.6
3.6
4.4
3.6
2.7
3.1
2.3
3.9
3.6
4.0
3.0
3.4
3.7
3.2
3.6
3.3
3.0
5.1
4.1
5.1
1.5
4.7
3.6
2.8
3.2
3.8
3.4
3.1
3.6
3.8
3.6
3.5
3.0
4.6
3.7
3.6
3.1
4.3


2.7
3.1
3.0
3.6
2.7
3.1
3.8
3.1
3.0
3.7
3.7
3.6
3.6
3.1
3.2
4.1
3.5
2.8
3.2
3.8
3.8
3.3
2.9
3.6
3.5
3.3
3.4
4.9
3.2
5.4
3.9
3.7
4.5
3.0
3.1
3.5
3.8
3.8
3.9
3.8
4.6
5.0
4.0
4.1
3.7
3.6
3.3
3.5
4.0
5.3
3.8
3.6
5.2
3.6
3.3
2.5
3.2
3.7
3.8
3.7
1.9
3.5
3.9
5.2
4.1
3.5
3.8
2.9
2.9
4.6
3.8
3.2
3.8
4.7
5.0
4.1
5.1
2.1
3.1
3.7
3.1
3.8
2.7
4.0
4.0
3.2
3.8
4.0
3.7
3.6
4.2
2.7
4.2
4.4
1.2
3.1
4.4
3.1
4.2
3.8
4.2
3.6
5.3
3.2
3.8
5.0
3.6
3.3
3.5
2.4
3.6
3.6
3.8
3.7
3.8
2.6
3.5
5.4
4.3
3.4
4.0
3.2
4.0
3.7
3.5
3.2
3.3
3.6
3.3
3.9
3.5
3.9
4.0
3.9
3.6
3.5
3.5
3.7
3.8
2.8
3.4
3.3
3.4
3.6
3.4
3.9
3.0
3.4
3.9
2.9
4.1
3.1
3.6
3.3
3.4
3.8
3.4
3.6
3.4
3.8
4.0
3.8
3.2
3.4
3.5
3.7
3.8
3.7
3.5
3.8
2.9
2.7
3.9
3.5
5.3
3.9
2.9
4.1
3.8
3.7
3.7
3.2
3.6
3.4
3.9
3.7
4.0
4.0
3.0
3.0
3.5
4.0
3.6
2.8
4.1
3.9
3.5
3.5
4.0
4.8
3.9
3.1
3.1
4.2
4.3
4.1
4.0
3.8
3.9
3.7
3.6
2.7
3.6
3.8
3.2
3.9
2.8
3.9
3.3
3.8
3.1
3.4
3.9
2.5
5.0
2.4
2.9
3.8
3.7
3.1
3.3
3.6
3.5
3.7
3.4
3.7
3.6
3.3
3.8
2.9
2.7
2.9
3.6
3.6
2.4
2.0
2.1
3.6
3.6
3.7


3.4
3.9
3.6
3.7
3.3
3.5
3.8
5.0
3.6
4.1
2.8
4.4
4.0
3.6
3.8
3.0
5.6
2.7
1.8
5.0
3.4
2.8
3.4
3.4
2.7
3.2
3.1
2.6
3.6
2.4
4.0
3.9
5.0
3.3
3.6
3.7
4.6
3.6
2.9
3.7
3.8
5.4
3.5
3.3
3.3
3.8
4.6
3.7
2.0
3.5
2.9
3.8
3.3
3.6
3.9
3.4
3.6
4.3
3.0
3.4
4.0
2.4
3.2
3.0
3.7
3.7
3.0
3.8
3.1
3.1
3.7
3.7
3.8
3.6
2.6
3.4
3.5
3.2
4.1
3.5
2.3
3.5
2.8
2.5
3.6
3.8
4.0
3.6
3.1
3.8
4.0
2.7
3.0
3.1
3.7
3.7
3.5
3.8
3.5
3.3
3.7
4.0
3.8
1.6
3.7
0.5
2.9
3.7
3.8
4.1
3.7
2.5
3.6
4.4
4.0
3.4
3.2
4.6
3.8
2.9
3.8
3.6
3.9
2.9
1.8
3.9
4.6
3.9
3.7
3.9
3.7
2.7
3.0
2.4
2.3
2.3
3.6
3.5
2.8
2.6
3.3
2.9
3.0
3.2
2.2
2.7
2.5
3.1
5.1
3.5
3.7
3.2
4.2
3.2
3.0
2.0
2.6
4.6
3.5
3.0
2.7
3.5
3.7
4.0
2.9
3.4
2.6
3.8
2.9
2.1
3.1
3.3
3.2
3.0
3.0
2.4
3.9
2.4
3.7
3.2
3.2
3.7
3.4
2.6
2.9
3.2
3.3
3.3
2.9
2.3
3.7
3.4
3.0
3.3
4.0
2.7
2.7
3.4
2.7
3.6
3.5
5.4
2.6
3.9
2.9
1.9
3.1
3.4
3.1
3.6
3.3
3.8
2.7
2.7
3.7
3.9
3.2
3.8
3.2
3.3
3.1
3.6
3.8
3.2
3.4
3.8
3.3
3.4
2.3
3.4
2.5
3.0
3.5
3.1
3.1
3.4
3.3
3.8
4.4
3.7
5.3
3.8
1.8
2.8
1.5
3.0
4.3
2.2
3.6
1.9


...

3.1
3.8
3.8
3.3
3.8
3.2
3.5
2.9
3.5
2.1
4.0
2.2
3.5
3.3
3.1
2.2
3.6
3.2
3.5
4.0
3.8
3.5
3.3
3.2
4.2
1.8
2.9
4.2
3.8
4.7
2.2
4.3
3.7
3.5
3.7
1.6
3.4
4.4
3.6
3.7
3.7
2.7
2.1
2.5
3.2
2.8
3.8
3.7
3.5
1.6
4.0
3.6
3.8
3.2
3.5
3.4
4.1
3.9
2.3
3.3
3.9
3.5
3.1
3.5
3.7
3.2
3.4
3.4
3.5
2.3
2.6
3.7
2.9
3.0
2.0
3.5
3.5
3.8
2.2
3.4
5.0
3.8
5.0
3.5
3.8
2.1
4.0
4.2
3.9
3.7
3.7
2.6
5.0
2.1
2.5
4.1
2.9
3.6
3.2
4.2
3.9
2.5
3.1
4.0
4.1
3.9
4.0
3.5
3.1
3.6
4.0
4.0
2.6
3.5
4.0
2.8
5.3
3.0
4.1
4.0
4.0
3.7
2.9
3.3
3.7
4.0
3.6
5.4
3.7
3.8
3.6
4.6
1.8
3.6
3.3
1.8
4.9
3.2
3.1
3.6
3.9
4.6
3.6
3.6
3.8
3.7
3.5
2.8
3.6
3.6
3.3
3.6
4.0
4.6
3.8
3.7
4.4
3.8
3.9
2.9
3.8
3.8
3.1
3.9
3.4
3.0
2.9
3.8
3.8
3.4
3.3
2.6
3.8
3.9
3.8
3.2
4.2
3.0
2.1
3.7
4.6
3.4
3.6
2.0
3.4
4.2
3.8
3.7
3.5
4.0
4.4
3.5
4.0
4.1
3.9
3.8
3.8
3.4
3.3
3.0
3.5
2.6
3.7
4.2
2.9
3.4
4.4
3.7
3.1
1.8
3.2
3.2
2.7
3.1
3.3
3.3
3.5
2.7
3.9
3.1
2.6
2.5
3.8
1.9
3.1
3.3
3.6
3.2
3.5
3.3
3.0
2.6
3.7
3.7
3.1
3.8
3.5
2.6
3.3
3.3
3.6
5.2
1.6
3.8
3.4
3.5
3.6
2.8
3.2
3.5


3.6
3.0
2.8
3.9
2.5
3.4
2.4
3.5
2.0
3.5
3.1
3.6
2.3
2.6
2.9
3.0
3.6
3.1
3.1
3.5
3.1
3.1
3.0
2.9
3.5
2.4
3.7
4.1
2.1
2.7
2.7
3.6
3.4
3.8
1.9
3.6
3.5
2.6
3.6
3.8
4.0
3.8
2.8
2.4
3.4
2.8
2.5
3.6
3.5
4.3
4.8
3.4
3.6
2.6
4.3
3.1
4.7
5.5
3.2
5.2
3.4
3.2
4.6
2.5
2.1
5.0
4.1
3.3
3.4
2.9
3.0
3.5
3.9
3.8
3.6
2.9
3.5
3.6
4.1
3.8
1.8
2.6
2.6
3.5
3.3
4.1
3.6
3.4
2.3
3.4
5.6
3.6
2.6
2.9
2.7
2.9
3.5
5.4
3.4
3.2
3.6
2.8
4.0
3.9
2.3
4.0
3.9
3.7
2.8
3.8
4.4
3.4
2.9
3.4
3.8
2.5
3.1
3.1
4.1
3.9
3.9
3.4
3.9
3.2
2.5
3.3
2.8
3.9
4.7
4.3
2.0
2.6
3.5
4.0
3.3
3.9
3.7
4.0
3.8
2.4
3.8
4.2
1.8
3.7
2.6
4.2
2.7
2.7
3.9
3.7
2.1
3.5
3.6
3.6
3.8
2.3
4.6
3.1
3.7
4.3
4.1
3.7
3.4
3.2
3.5
1.8
2.4
3.4
3.4
4.3
4.0
4.0
2.8
3.2
3.9
3.8
3.2
3.5
3.4
4.1
3.4
4.1
4.3
4.0
4.0
3.5
3.9
3.7
4.1
2.5
3.8
4.4
3.4
3.5
3.6
3.6
3.7
4.0
3.5
3.8
3.5
3.5
3.8
3.8
4.1
3.4
3.6
3.4
3.3
3.0
3.8
3.9
3.2
3.2
4.1
4.0
2.8
3.6
4.1
3.8
3.9
3.1
4.1
4.0
3.2
2.3
3.3
3.4
3.6
4.0
3.3
2.9
1.5
4.6
2.9
3.0
3.1
3.9
3.1
2.8
3.0
3.3
2.7
2.5
3.4
3.0
3.7
2.1
2.7
2.9


4.0
3.7
4.0
4.0
3.9
3.0
4.0
4.1
4.0
2.8
3.5
3.3
3.7
3.7
2.7
2.8
3.7
3.8
3.4
3.4
4.7
5.4
3.8
3.0
3.5
2.9
2.5
3.6
3.4
3.7
3.6
3.3
3.5
4.1
3.8
3.7
3.7
3.1
3.6
3.5
3.6
3.4
3.5
3.8
3.6
3.9
3.3
3.6
3.0
4.3
3.2
4.0
3.7
3.1
3.2
3.5
4.6
3.6
3.5
4.0
2.6
3.5
4.0
3.1
3.7
3.2
3.9
1.8
4.0
3.1
5.1
3.5
3.9
3.4
3.1
4.7
1.8
3.5
3.4
3.3
3.4
3.7
3.1
3.0
3.0
3.4
3.5
3.7
3.8
2.8
3.4
4.2
3.6
3.8
3.8
3.0
3.8
2.9
3.0
2.6
4.0
5.3
1.8
3.2
1.8
2.1
2.2
2.5
5.1
4.1
4.0
4.3
4.0
3.3
4.0
3.8
2.5
3.6
3.7
3.5
3.9
3.9
2.9
3.7
4.3
3.2
5.0
2.5
4.4
3.5
2.9
4.0
3.3
3.4
3.4
3.3
3.8
5.6
3.4
3.7
3.1
3.1
2.5
2.7
3.4
4.3
3.5
4.0
3.7
3.8
2.4
4.1
4.0
3.7
3.9
3.0
3.1
3.2
3.2
2.4
2.6
2.9
3.6
3.6
3.6
3.8
3.9
2.2
4.1
3.7
3.4
3.0
4.1
3.4
2.9
3.5
3.1
3.5
4.2
3.5
3.8
1.9
3.5
3.9
1.6
3.7
3.9
3.4
3.8
3.0
3.7
3.2
3.5
3.6
3.3
3.5
4.4
3.2
4.2
5.8
3.2
2.7
4.2
2.6
3.8
2.7
4.6
3.9
3.8
4.1
3.7
3.3
3.4
2.6
2.8
3.9
3.8
2.4
3.5
3.3
3.6
2.7
3.4
3.2
2.9
3.7
3.6
3.6
3.6
4.1
1.5
3.6
3.7
3.7
2.3
2.4
3.8
3.5
3.6
4.4
2.6
1.9
3.4
3.8
4.0
3.9
3.7
3.8
3.5
3.4


3.2
3.5
3.2
3.6
2.9
3.2
2.5
3.7
3.5
2.6
3.4
3.7
3.5
3.4
3.8
4.5
3.1
3.4
3.9
3.6
3.4
4.4
2.3
3.7
3.0
3.8
1.6
4.1
3.2
3.4
3.6
1.9
3.7
3.2
3.6
3.6
3.5
4.0
3.9
2.4
2.8
3.5
3.8
3.5
3.7
3.3
3.4
3.0
4.1
4.0
2.4
3.9
4.1
1.7
2.4
3.3
1.7
3.9
3.3
3.0
2.9
2.6
2.3
3.7
3.2
3.9
2.6
4.1
3.2
2.8
3.3
3.3
2.9
3.2
2.4
3.9
3.3
3.2
4.1
3.7
2.9
3.9
3.4
4.0
2.9
3.5
3.9
4.1
2.6
2.4
3.2
3.8
3.1
2.7
3.2
3.3
2.1
3.0
3.5
4.0
2.8
2.6
2.3
3.2
3.2
2.9
5.4
3.1
3.2
2.6
2.3
2.6
3.8
2.9
3.1
5.3
3.9
2.8
3.7
4.5
3.0
2.5
3.5
3.3
1.9
3.7
3.8
3.7
1.6
3.5
3.3
4.4
3.7
3.9
3.7
3.7
3.5
5.3
2.2
3.4
3.4
3.5
2.3
3.5
4.1
3.0
3.1
4.2
3.1
3.4
2.4
2.3
3.6
5.3
3.7
3.2
4.6
3.0
3.3
3.0
4.1
3.3
2.6
3.1
3.8
4.0
4.2
4.0
2.9
2.1
2.9
3.4
4.1
4.0
3.7
2.7
3.8
2.6
3.9
3.5
3.0
3.4
3.3
3.1
2.4
3.6
3.6
3.8
5.4
3.5
3.3
5.0
3.6
4.2
3.1
3.1
5.0
3.4
3.7
2.6
3.4
3.1
3.1
3.1
2.9
3.2
2.8
4.5
3.5
3.9
2.8
3.7
2.6
3.3
2.9
2.2
3.7
4.3
2.9
2.9
3.9
3.2
3.3
2.7
3.4
3.3
3.3
3.5
2.9
4.0
3.5
2.7
4.0
2.5
3.7
3.7
3.6
2.4
3.5
5.6
3.5
4.1
3.1
4.2
5.3
3.4
1.8
3.4
3.3
4.2


4.3
4.0
3.8
5.0
2.9
2.1
4.0
3.2
3.7
3.0
3.4
3.8
2.9
5.3
2.2
3.4
2.9
3.0
2.1
2.8
3.7
3.0
3.0
4.3
3.0
2.7
2.8
3.0
2.6
2.9
3.4
3.1
2.1
2.2
3.7
2.9
3.7
3.2
3.7
5.3
2.7
2.4
5.6
1.9
3.7
2.8
2.5
2.5
2.5
2.5
2.5
2.5
2.5
2.5
2.5
2.5
2.5
2.5
2.5
2.5
